In [28]:
# Bibliotecas
import numpy as np
import math



In [29]:
u = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
     0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
v = [4.74, 5.12, 5.39, 5.49, 5.43, 5.21, 4.85,
     4.42, 3.97, 3.56, 3.26, 3.11, 3.13, 3.31, 3.63]


def f(x):
    """Funición a evaluar"""
    a = 0
    for i in range(0, 15):
        aux = x[0]*math.sin(x[1]*u[i])+x[2]-v[i]
        a += aux**2
    return a


def jHess(x):
    jH = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        jH += np.outer(j, j)
    return jH


def jGrad(x):
    jG = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        jG += j*fxerror(x, i)
    return jG

def fxerror(x, i):
    return x[0]*(math.sin(x[1]*u[i]))+x[2]-v[i]


def jacobian(x, i):
    return np.array([math.sin(x[1]*u[i]),
                     u[i]*x[0]*math.cos(x[1]*u[i]),
                     1
                     ])


In [30]:
# gauss newton

def gaussNewton(x0, tol=1e-4):
    k = 0
    print(k, x0)
    while np.linalg.norm(jGrad(x0)) > tol:
        a = jHess(x0)
        s =  np.dot(np.linalg.inv(a), -jGrad(x0))
        x0 = x0 + s
        k = k+1
        print(k,  x0, f(x0))
    return x0

     

In [31]:
#Evaluar Gauss Newton
x0 = np.array([1.0, 2.0, 1.3])
# grad = jHess(x0)
gn = gaussNewton(x0)


0 [1.  2.  1.3]
1 [-0.93568029  3.49735647  5.39809436] 53.37613144510951
2 [1.14538958 3.08262573 4.32987585] 3.2740392361556117
3 [0.8072063  3.95417301 4.54756344] 2.3393236847142225
4 [1.18849459 3.72984109 4.30197552] 0.03666322402337587
5 [1.19729029 3.80013066 4.29694345] 0.0002045916233588216
6 [1.19974186 3.79842054 4.29544624] 0.00010264046529553048


In [32]:
def jHessFull(x):
    jH = 0
    for i in range(0, 15):
        j = jacobian(x, i)
        t = tk(x, i)
        fxe = fxerror(x, i)
        jH += np.outer(j, j) + np.dot(t, fxe)
    return jH

def tk(x, i):
    return np.array([
            [0, u[i]*math.cos(x[1]*u[i]), 0],
            [u[i]*math.cos(x[1]*u[i]), -1*x[0]*(u[i]**2)*math.sin(x[1]*u[i]), 0],
            [0, 0, 0]
        ])


def levenbergMarquardtMC(x0, tol=1e-6):
    k = 0
    mu = 1e-4
    v = 10
    I = np.identity(x0.size)
    print(k, x0, f(x0))
    while np.linalg.norm(jGrad(x0)) >= tol:
        mu = mu/v
        while True:
            delta = np.dot(np.linalg.inv(jHess(x0) + mu*I), jGrad(x0))
            x1 = x0 - delta
            mu = mu * v
            # k = k+1
            print(k,  x0, round(f(x0), 6), x1, round(f(x1), 6), mu)
            if (f(x1) < f(x0)):
                x0 = x1
                break
        k = k+1
    return x0


In [33]:
x0 = np.array([1.0, 2.0, 1.3])
# jHessFull(x0)
levenbergMarquardtMC(x0)



0 [1.  2.  1.3] 94.15754849549766
0 [1.  2.  1.3] 94.157548 [-0.93562588  3.49733997  5.39805025] 53.372779 0.0001
1 [-0.93562588  3.49733997  5.39805025] 53.372779 [1.14538051 3.08256099 4.32988084] 3.274627 0.0001
2 [1.14538051 3.08256099 4.32988084] 3.274627 [0.80712493 3.95419439 4.54761399] 2.340253 0.0001
3 [0.80712493 3.95419439 4.54761399] 2.340253 [1.18849118 3.72981044 4.30197739] 0.036695 0.0001
4 [1.18849118 3.72981044 4.30197739] 0.036695 [1.19728804 3.80013201 4.29694481] 0.000205 0.0001
5 [1.19728804 3.80013201 4.29694481] 0.000205 [1.19974185 3.79842054 4.29544624] 0.000103 0.0001
6 [1.19974185 3.79842054 4.29544624] 0.000103 [1.19974328 3.79842307 4.29544531] 0.000103 0.0001


array([1.19974328, 3.79842307, 4.29544531])

In [34]:
# newtonNDim
def newtonNDim(x0, tol=1e-4):
    k = 0
    while np.linalg.norm(jGrad(x0)) > tol:
        x0 = x0 - np.dot(np.linalg.inv(jHessFull(x0)), jGrad(x0))
        k = k+1
        print(k, f(x0), x0)
    return x0


In [35]:
x0 = np.array([1.0, 2.0, 1.3])
newtonNDim(x0)


1 54.017252177768384 [3.99580259 1.31260472 1.43307402]
2 8.564631058236923 [-1.30516037  1.55750348  5.12558769]
3 1274.6327100242845 [ 9.86327327  4.93423432 -3.2325974 ]
4 942.0638629707175 [-0.4929472   0.52657729 12.38857608]
5 9.84720393540005 [-0.46540121  0.50124599  4.48503307]
6 26.760342877078983 [ 1.01592982 -0.53718744  3.59041409]
7 5.887812686621256 [ 1.31759996 -0.69634203  4.95010312]
8 4.385025031366485 [ 3.10649552 -0.99218937  6.10389088]
9 3.4176903027688565 [ 2.52439716 -1.02930125  5.99144875]
10 3.125172208826077 [ 2.70551128 -0.81162292  5.90479549]
11 3.002247617753201 [ 2.91097729 -0.739984    5.86287679]
12 2.9569746664736765 [ 3.22339738 -0.62415701  5.82879228]
13 2.9220905985036865 [ 3.42279675 -0.59167454  5.82282501]
14 2.9199622970592403 [ 3.8478207  -0.49635976  5.79664164]
15 2.893917399085221 [ 3.99653985 -0.48904989  5.79901873]
16 3.007176060307242 [ 4.80749608 -0.36879838  5.76958427]
17 2.878508171965549 [ 5.01466233 -0.38004975  5.78751216]
18 

array([-8.50610872e-07,  1.91498460e+00,  4.30800072e+00])

In [36]:
def broyden(x0, tol=1e-4):
    k = 0
    a = np.identity(x0.size)
    while np.linalg.norm(jGrad(x0)) > tol:
        s = np.dot(np.linalg.inv(a), -jGrad(x0))
        x1 = x0 + s
        y = jGrad(x1)-jGrad(x0)
        x0 = x1
        a = a + np.outer(y-np.dot(a, s), s)/np.dot(s, s)
        print(k, x0, f(x0))
        k = k+1
        # print(a)
    return x0


In [37]:
x0 = np.array([1.0, 2.0, 1.3])
broyden(x0)


0 [23.77604705 -3.00083665 36.43266619] 14824.497368337488
1 [ 20.81878505 217.74138233  21.1075891 ] 7345.838269755365
2 [ 6.13694635 18.05298568  1.3087227 ] 410.5857832352874
3 [-41.48535425  24.16787519  33.14671078] 26934.0928902792
4 [-0.08344339 14.41339317  3.94029276] 13.92964036699118
5 [ 0.22627734 16.93943554  4.19983842] 12.283463252150877
6 [ 0.042184   16.94068678  4.30048947] 11.83612699243576
7 [ 0.03368808 16.95462717  4.30761099] 11.835199985583483
8 [ 0.03272593 16.95122225  4.30766251] 11.83509555748199
9 [ 0.03287503 16.95094815  4.30765832] 11.83508774745636
10 [ 0.0328338  16.95165719  4.30765734] 11.835107760380543
11 [-1.69370496e-02  1.78102335e+01  4.30648901e+00] 11.838911436846008
12 [ 0.05106744 16.68206781  4.30720214] 11.828409943344866
13 [-0.06327521 18.4221889   4.30956786] 11.82941231885939
14 [ 0.03261956 16.91686799  4.3087573 ] 11.834173258320375
15 [ 0.03215375 16.96713356  4.30784437] 11.835544407148246
16 [ 0.04408824 16.75793199  4.30790875] 

array([3.21350948e-06, 1.54769101e+01, 4.30799859e+00])

In [38]:
#descenso de gradiente

def dirgrad(x):
    vgrad = jGrad(x)
    magGrad = np.sqrt(vgrad.dot(vgrad))
    p = -vgrad/magGrad
    return p


def phiAlpha(x0, alpha, p):
    paX = x0 + p * alpha
    return f(paX)


def phipAlpha(x0, alpha, p):
    x = x0 + alpha * p
    vgrad = jGrad(x)
    return (np.dot(vgrad, p))


def phipp(x0, alpha, p):
    x = x0 + alpha * p
    ahess = jHessFull(x)
    return np.dot(np.dot(ahess, p), p)





def exhaustivoRefinado(p, xini, alpha=0, h=0.1, tol=1e-9):
    """Busqueda de minimo con metodo exhaustivo refinado. puedes cambiar el paso
    Retorna f(a) y alpha
    """
    k = 0
    while h > tol:
        while phiAlpha(xini, alpha+h, p) < phiAlpha(xini, alpha, p):
            alpha = alpha + h
            fnow = phiAlpha(xini, alpha, p)
            # print(k, h, fnow)
            k += 1
        alpha = alpha-h
        h = h / 10
    return alpha


def gradDescent(x0, k=0, tol=1e-6):
    print("k, x^(k), f(x^(k), pk, θ,")
    op = dirgrad(x0)
    while np.linalg.norm(jGrad(x0)) >= tol:
        p = dirgrad(x0)
        alpha = exhaustivoRefinado(p, x0)
        # print(f"a: {alpha}")
        x0 = x0 + alpha*p
        if k >= 1:
            angulo = np.arccos(np.dot(op, p))
            op = p
            print(f"{k}, {x0}, {f(x0)}, {p},  {round(np.degrees(angulo),6)} ")
        else:
            print(f"{k}, {x0}, {f(x0)}, {p},  - ")

        k = k+1
    return x0


In [39]:
x0 = np.array([1.0, 2.0, 1.3])
gradDescent(x0)


k, x^(k), f(x^(k), pk, θ,
0, [1.94914572 1.79160024 2.76408284], 18.74744377395421, [ 0.54013837 -0.11859581  0.83317799],  - 
1, [1.88687047 2.38495533 2.88891415], 11.5212870332878, [-0.10216867  0.97345741  0.20479804],  89.999998 
2, [1.95980422 2.33139573 3.1798815 ], 9.384534187552484, [ 0.23935232 -0.17577069  0.95489011],  89.999998 
3, [1.88242504 2.5826336  3.24552378], 8.294732937054158, [-0.28557885  0.92722908  0.24226217],  89.999994 
4, [1.83521421 2.51673855 3.44207696], 7.397429706178855, [-0.22205081 -0.30992996  0.92446572],  89.999999 
5, [1.59940207 2.84449563 3.49531798], 6.350047855403266, [-0.57901071  0.80477135  0.13072746],  89.999994 
6, [1.54715709 2.764177   3.75836703], 4.889099039147942, [-0.18661846 -0.28689721  0.93960819],  89.999998 
7, [1.04683873 3.42038571 3.85936219], 2.8392608494774683, [-0.60182065  0.78933732  0.12148459],  89.999995 
8, [1.13580647 3.43235639 4.22231967], 0.7577182034806647, [0.23794913 0.03201624 0.9707498 ],  89.999997 
9, 

array([1.19974324, 3.79842299, 4.29544531])

In [40]:
def gD(x0):
    p = dirgrad(x0)
    alpha = exhaustivoRefinado(p, x0)
    # TODO: buscar alpha con newton para mayor precisión ?
    x0 = x0 + alpha*p
    return x0


def forsyte(x0, k=0, m=0, tol=1e-2):
    """Algoritmo de forsyte."""
    print("k, x ^ (k), f(x ^ (k), pk")
    while np.linalg.norm(jGrad(x0)) >= tol:
        x1 = gD(x0)
        x2 = gD(x1)
        x3 = gD(x2)
        y = x3
        d = (y - x0)/np.linalg.norm(y - x0)
        alpha = exhaustivoRefinado(d, x0)
        # TODO: buscar alpha con newton para mayor precisión ?
        # print(f"alpha: {alpha}")
        x0 = x0 + alpha*d
        # itTime = timer()
        print(f"{k}, {x0}, {f(x0)}, {d} ")
        k = k + 1
    return x0


In [41]:
x0 = np.array([1.0, 2.0, 1.3])
forsyte(x0)


k, x ^ (k), f(x ^ (k), pk
0, [1.98768275 2.34102147 3.23448466], 9.323111447940448, [0.4492232  0.15510523 0.87985276] 
1, [0.92034716 3.72185805 4.0734375 ], 1.3464740630673415, [-0.5511876   0.71308407  0.43324743] 
2, [1.19160467 3.7793176  4.29915715], 0.0035638437979520383, [0.75868704 0.16071008 0.63132103] 
3, [1.19929015 3.79788801 4.29452828], 0.00011683237100980668, [ 0.3726457   0.90042314 -0.22444008] 
4, [1.19972604 3.79838614 4.29545396], 0.00010265425538361158, [0.38303801 0.4377257  0.81343598] 


array([1.19972604, 3.79838614, 4.29545396])

In [42]:
# Gradientes Conjugados NL
def newton(xo, p, ao, itmax=100, tol=1e-12):
    k = 0
    # ak = 0
    while abs(phipAlpha(xo, ao, p)) > tol:
        phiap = phipAlpha(xo, ao, p)
        phiapp = phipp(xo, ao, p)
        ak = ao - phiap/phiapp
        # print(f"\t {k}, {ak}")
        k = k+1
        ao = ak
        if k >= itmax:
            print("Iteraciones exedidas")
            break
    return ak


def gradientesConjugados(x0, flavor="FR", k=0, tol=1e-6):
    beta = 0
    p = -jGrad(x0)
    print("k, fx, x0, alpha, beta, ")
    while np.linalg.norm(jGrad(x0)) >= tol:
        alpha = newton(x0, p, 0)
        xi = x0 + alpha*p
        gradxi = jGrad(xi)
        gradx0 = jGrad(x0)
        if flavor == "FR":
            beta = np.dot(gradxi, gradxi)/np.dot(gradx0, gradx0)
        elif flavor == "PR":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
        elif flavor == "PR+":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
            if beta < 0:
                beta = 0
        elif flavor == "HS":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                np.dot((gradxi - gradx0), p)
        p = -gradxi + beta*p
        print(k, f(x0), xi, alpha, beta)
        x0 = xi
        k += 1
    return x0


In [44]:
x0 = np.array([1.0, 2.0, 1.3])
gradientesConjugados(x0,flavor="FR")

k, fx, x0, alpha, beta, 
0 94.15754849549766 [1.94914573 1.79160024 2.76408285] 0.04167297878840474 0.060878374343717706
1 18.747443773954203 [2.02662322 4.14362042 3.78633343] 0.23942760378580574 1.0382856625602845
2 10.399532329801419 [1.8379291  4.36389756 4.17489594] 0.03155052291676524 0.7463486246789234
3 6.822440548056421 [1.51220548 4.29250593 4.53774657] 0.033573948870974674 0.6738290215457408
4 3.994497845539156 [1.22713232 4.06238271 4.65127131] 0.03087289892251813 0.5548701086255411
5 2.3052148287807377 [1.00300198 3.7866028  4.52150219] 0.04259656196489717 0.4461148244815673
6 1.0774794719304255 [0.97069997 3.67944351 4.32426414] 0.04161969424279546 0.2816530262639268
7 0.5032591487096914 [1.05202087 3.674519   4.22127262] 0.05233542691105991 0.5734709026011945
8 0.29711358788433234 [1.18352891 3.70290328 4.20418379] 0.06517048905782614 0.660791964289599
9 0.14986331399089345 [1.25610216 3.73937625 4.25497767] 0.052075024906906485 0.4277116631468989
10 0.07205523071623406 

array([1.19974323, 3.79842312, 4.29544529])